In [5]:




#-------------------------------------------------------------------
#----------------------   PROJET VELO PARIS   ----------------------
#-------------------------------------------------------------------

# Données issues de l’Open Data de la Ville de Paris :
# https://opendata.paris.fr/explore/dataset/comptage-velo-donnees-compteurs/
# Période : du 01/09/2024 au 31/08/2025

# === 1. Importation des librairies ===
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from fpdf import FPDF
import plotly.graph_objects as go
import geopandas as gpd
import requests
import json

# === 2. Chargement des données ===
df = pd.read_csv("comptage-velo-donnees-compteurs-annee.csv", sep=";")

# Copie dans une base SQLite en mémoire (utile pour tester des requêtes SQL rapidement)
conn = sqlite3.connect(":memory:")
df.to_sql("Compteurs", conn, index=False, if_exists="replace")

# === 3. Nettoyage des données ===
# On sélectionne uniquement les colonnes utiles
df = df[[
    "Date et heure de comptage",
    "Identifiant du site de comptage",
    "Identifiant du compteur",
    "Nom du site de comptage",
    "Comptage horaire",
    "Coordonnées géographiques"
]]

# Suppression des lignes incomplètes
df = df.dropna()

# Suppression des capteurs défectueux (moyenne = 0 vélos)
defecteux = df.groupby("Identifiant du site de comptage").agg(
    Moyenne_velo=("Comptage horaire", "mean")
).reset_index()
sites_defectueux = defecteux[defecteux["Moyenne_velo"] == 0]["Identifiant du site de comptage"].tolist()
df = df[~df["Identifiant du site de comptage"].isin(sites_defectueux)]

# Nettoyage des adresses (suppression des numéros, préfixes "Face au", "Totem")
df["Nom du site de comptage"] = (
    df["Nom du site de comptage"]
    .str.replace(r"^(Face\s*au\s*\d+\s*)", "", regex=True)
    .str.replace(r"^(Totem\s*\d+\s*)", "", regex=True)
    .str.replace(r"^\d+\s*", "", regex=True)
    .str.strip()
)

# Conversion en datetime UTC → suppression fuseau horaire
df["Date et heure de comptage"] = pd.to_datetime(df["Date et heure de comptage"], errors="coerce", utc=True)
df["Date et heure de comptage"] = df["Date et heure de comptage"].dt.tz_convert(None)

# Création de variables dérivées (heure, jour, mois)
df["Heure"] = df["Date et heure de comptage"].dt.hour + 1
df["Jour"] = df["Date et heure de comptage"].dt.day_name()

# Traduction des jours en français
jours_fr = {
    "Monday": "Lundi", "Tuesday": "Mardi", "Wednesday": "Mercredi",
    "Thursday": "Jeudi", "Friday": "Vendredi", "Saturday": "Samedi", "Sunday": "Dimanche"
}
df["Jour"] = df["Jour"].map(jours_fr)

# Extraction latitude/longitude
df[["lat", "lon"]] = df["Coordonnées géographiques"].str.split(",", expand=True).astype(float)

# Création des variables mois
df["Mois"] = df["Date et heure de comptage"].dt.month_name()
df["Mois_num"] = df["Date et heure de comptage"].dt.month  # utile pour trier

# Nombre de jours uniques par mois (sert à normaliser les moyennes)
jours_par_mois = (
    df.groupby("Mois_num")["Date et heure de comptage"]
      .apply(lambda x: x.dt.date.nunique())
      .reset_index(name="Nb_jours")
)

# Vérification rapide des données
print(" Vérification doublons et valeurs manquantes")
print("Nombre de doublons :", df.duplicated().sum())
print("Valeurs manquantes restantes :")
print(df.isnull().sum())

# === 4. Analyse des données ===

# 10 sites les moins fréquentés
# Non utilisé dans le pdf
moyenne_bot_site = (
    (df.groupby("Nom du site de comptage")["Comptage horaire"].sum() / 365)
    .reset_index().round(1)
    .sort_values(by="Comptage horaire", ascending=True).head(10)
)

# 10 sites les plus fréquentés
# Non utilisé dans le pdf
moyenne_top_site = (
    (df.groupby("Nom du site de comptage")["Comptage horaire"].sum() / 365)
    .reset_index().round(1)
    .sort_values(by="Comptage horaire", ascending=False).head(10)
)

# Moyennes par heure
moyenne_heure = (
    (df.groupby("Heure")["Comptage horaire"].sum() / 365)
    .reset_index().round(1)
    .sort_values(by="Heure")
)

# Moyennes par jour
moyenne_jour = (
    (df.groupby("Jour")["Comptage horaire"].sum() / 52)
    .reset_index().round(1)
    .sort_values(by="Comptage horaire", ascending=False)
)

# Moyennes par mois (corrigées par nb de jours du mois)
somme_mois = df.groupby("Mois_num")["Comptage horaire"].sum().reset_index(name="Total_velos")
moyenne_mois = somme_mois.merge(jours_par_mois, on="Mois_num")
moyenne_mois["Moyenne_jour"] = (moyenne_mois["Total_velos"] / moyenne_mois["Nb_jours"]).round(1)
# Traduction des mois en français
mois_fr = {
    "January": "Janvier","February": "Février","March": "Mars","April": "Avril",
    "May": "Mai","June": "Juin","July": "Juillet","August": "Août",
    "September": "Septembre","October": "Octobre","November": "Novembre","December": "Décembre"
}
moyenne_mois["Mois"] = moyenne_mois["Mois_num"].apply(lambda x: pd.to_datetime(str(x), format="%m").month_name())
moyenne_mois["Mois"] = moyenne_mois["Mois"].map(mois_fr)


# Pic du matin et du soir
pic_matin = moyenne_heure.idxmax() if moyenne_heure.index[0] < 12 else "Non défini"
pic_soir = moyenne_heure.idxmax() if moyenne_heure.index[0] >= 12 else "Non défini"

# Jours avec le plus et le moins de trafic
moyenne_annee = df.groupby("Jour")["Comptage horaire"].sum() / 365
jour_max = moyenne_annee.idxmax()
jour_min = moyenne_annee.idxmin()

# Mois avec le plus et le moins de trafic
# Assurez-vous que 'Moyenne_jour' dans moyenne_mois est déjà calculée
mois_max = moyenne_mois.loc[moyenne_mois["Moyenne_jour"].idxmax(), "Mois"]
mois_min = moyenne_mois.loc[moyenne_mois["Moyenne_jour"].idxmin(), "Mois"]

# Trafic moyen journalier
trafic_moyen = int(df["Comptage horaire"].sum() / 365)





# === 5. Visualisation des données ===
# (barres, courbes et carte → images .png pour le PDF)


# Graphique bar des 10 sites ayant la plus petite moyenne journalière
# Non utilisé dans le pdf
fig = px.bar(
    moyenne_bot_site, 
    x="Nom du site de comptage", 
    y="Comptage horaire", 
    title="Top 10 des sites par plus faibles moyennes de vélos", 
    color="Comptage horaire",
    color_continuous_scale=[
        (0, "#6d93d1"),   
        (0.5, "#48628c"), 
        (1, "#222d45")    
    ]
)
fig.update_layout(
    template="plotly",
    title_x=0.5,  # centre le titre
    xaxis_title="Site de comptage",
    yaxis_title="Nombre moyen de vélos"
)
fig.write_image("bot10.png")


# Graphique bar des 10 sites ayant la plus grande moyenne journalière
# Non utilisé dans le pdf
fig2 = px.bar(
    moyenne_top_site, 
    x="Nom du site de comptage", 
    y="Comptage horaire", 
    title="Top 10 des sites par plus fortes moyennes de vélos", 
    color="Comptage horaire",
    color_continuous_scale=[
        (0, "#6d93d1"),   
        (0.5, "#48628c"), 
        (1, "#222d45")    
    ]
)
fig2.update_layout(
    template="plotly",
    title_x=0.5,
    xaxis_title="Site de comptage",
    yaxis_title="Nombre moyen de vélos"
)
fig2.write_image("top10.png")


# Graphique line des moyennes journalières selon l'heure
fig3 = px.line(
    moyenne_heure, 
    x="Heure", 
    y="Comptage horaire", 
    title="Moyenne des vélos par heure",
    markers=True
)
fig3.update_traces(line_color="#48628c", line_width=3.5)

fig3.update_xaxes(
    tickmode="linear",
    dtick=1,   # 1 = un tick par unité
    tick0=1    # commence à 1
)

fig3.update_layout(
    template="plotly",
    title_x=0.5,
    xaxis_title="Heure de la journée",
    yaxis_title="Nombre moyen de vélos"
)
fig3.write_image("moyenne_heure.png")


# Graphique bar des moyennes journalières selon le jour de la semaine
fig4 = px.bar(
    moyenne_jour, 
    x="Jour", 
    y="Comptage horaire", 
    title="Moyenne journalière des vélos par jour de la semaine", 
    color="Comptage horaire",
    color_continuous_scale=[
        (0, "#6d93d1"),   
        (0.5, "#48628c"), 
        (1, "#222d45")    
    ]
)
fig4.update_layout(
    template="plotly",
    title_x=0.5,
    xaxis_title="Jour de la semaine",
    yaxis_title="Nombre moyen de vélos"
)
fig4.write_image("moyenne_jour.png")


# Graphique bar des moyennes journalières selon les mois
fig5 = px.bar(
    moyenne_mois, 
    x="Mois", 
    y="Moyenne_jour", 
    title="Moyenne journalière des vélos par mois", 
    color="Moyenne_jour",
    color_continuous_scale=[
        (0, "#6d93d1"),   
        (0.5, "#48628c"), 
        (1, "#222d45")    
    ]
)
fig5.update_layout(
    template="plotly",
    title_x=0.5,
    xaxis_title="Mois",
    yaxis_title="Nombre moyen de vélos"
)
fig5.write_image("moyenne_mois.png")



# Maps du nombres de vélo selon les compteurs à 17h
int_heure = df.groupby(["Nom du site de comptage", "lat", "lon", "Heure"]).agg(
    Velos=("Comptage horaire", "sum")
).reset_index()

#Filtrer pour 17h
int_17h = int_heure[int_heure["Heure"] == 17]

#Valeurs min/max pour l'échelle de couleur
vmin = int_17h["Velos"].min()
vmax = int_17h["Velos"].max()

fig = px.scatter_map(
    int_17h,
    lat="lat",
    lon="lon",
    size="Velos",
    color="Velos",
    hover_name="Nom du site de comptage",
    size_max=35,
    color_continuous_scale="Viridis",
    range_color=[vmin, vmax],
    zoom=12,
    map_style="carto-positron"  
)

# Ajustements du rendu
fig.update_traces(
    marker=dict(
        opacity=0.7,  
        sizemode="area"
    )
)

# Mise en page carrée et centrée
fig.update_layout(
    width=700,
    height=700,
    title="Trafic cycliste à Paris - 17h",
    title_x=0.5,  # centrer le titre
    margin=dict(l=20, r=20, t=50, b=20),
    coloraxis_colorbar=dict(title="Nombre de vélos"),
    map=dict(center={"lat": 48.8566, "lon": 2.3441}, zoom=10.8)  # centrage sur Paris
)

# Affichage et export
fig.write_image("carte_17h.png", scale=2)






# === 6. Exportation des résultats ===
# + génération d’un PDF structuré avec FPDF
# (page de garde, sommaire, résumé exécutif, graphiques, conclusion)



kpis = {
    "Trafic moyen journalier": f"{trafic_moyen} vélos",
    "Pic du matin": "8h - 9h",   
    "Pic du soir": "17h - 19h",
    "Jour le plus chargé": jour_max,
    "Jour le moins chargé": jour_min,
    "Mois le plus chargé": mois_max,
    "Mois le moins chargé": mois_min
}



class PDF(FPDF):
    def footer(self):
        if self.page_no() != 1:
            self.set_y(-15)
            self.set_font("Helvetica", 'I', 8)
            self.cell(0, 10, f"Rapport - Analyse des Comptages de Vélos à Paris | Page {self.page_no()-1}", align="C", ln=1)

pdf = PDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# --- PAGE DE GARDE ---
# Logo en haut (centré)
pdf.image("Ville_de_Paris_logo.svg.png", x=80, y=10, w=50)  # Ajuste x, y et w selon la taille souhaitée

pdf.set_font("Helvetica", 'I', 12)
pdf.ln(60)  # Décalage après le logo
pdf.cell(0, 10, "Auteur : Barnabé Willenbucher - Data Analyst Freelance", ln=1, align="C")

pdf.set_font("Helvetica", 'B', 22)
pdf.set_y(100)
pdf.cell(0, 15, "Rapport - Analyse des Comptages de Vélos à Paris", ln=1, align="C")
pdf.ln(20)

pdf.set_font("Helvetica", 'I', 14)
pdf.cell(0, 10, "Données : Ville de Paris (Open Data - Année 2024)", ln=1, align="C")
pdf.ln(40)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "Ce rapport présente une étude approfondie des flux de vélos à Paris.\n\n"
    "À partir des données issues des compteurs automatiques, nous mettons en évidence "
    "les principales tendances d'utilisation du vélo : horaires de pointe, jours les plus actifs "
    "et variations saisonnières. L'objectif est de fournir des éléments concrets pour "
    "comprendre les comportements cyclistes et éclairer les décisions en matière de mobilité urbaine.",
    align="C"
)

# --- TABLE DES MATIÈRES ---
pdf.add_page()
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "Sommaire", ln=1, align="C")
pdf.ln(8)

pdf.set_draw_color(180, 180, 180)
pdf.set_line_width(0.3)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.set_font("Helvetica", '', 12)
entries = [
    ("Résumé exécutif", 3),
    ("1. Analyse par heure", 4),
    ("2. Analyse par jour de la semaine", 5),
    ("3. Analyse par mois", 6),
    ("4. Analyse géographique", 7),
    ("Conclusion", 8),
]

# Affichage titres à gauche et pages à droite
for title, page in entries:
    pdf.cell(0, 10, f"{title}", ln=0, align="L")
    pdf.cell(0, 10, str(page), ln=1, align="R")
    
pdf.ln(8)

# --- RÉSUMÉ EXÉCUTIF ---
pdf.add_page()
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "Résumé exécutif", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

# KPIs encadrés
pdf.set_fill_color(245, 245, 245)
pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)

kpis = {
    "Trafic moyen journalier": f"{trafic_moyen} vélos",
    "Pic du matin": "8h - 9h",
    "Pic du soir": "17h - 19h",
    "Jour le plus chargé": jour_max,
    "Jour le moins chargé": jour_min,
    "Mois le plus chargé": mois_max,
    "Mois le moins chargé": mois_min
}

pdf.set_font("Helvetica", 'B', 12)
for kpi, value in kpis.items():
    pdf.cell(60, 10, f"{kpi} :", border=1, fill=True)
    pdf.set_font("Helvetica", '', 12)
    pdf.cell(0, 10, value, border=1, ln=1, fill=True)
    pdf.set_font("Helvetica", 'B', 12)

pdf.ln(8)
pdf.set_font("Helvetica", 'B', 14)
pdf.cell(0, 10, "Précisions :", ln=1)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "- Le trafic moyen journalier ne reflète pas la réalité car les mêmes vélos ont pu être comptabilisés plusieurs fois.\n"
    "- Les pics d'utilisation se situent à 8h et entre 17h et 19h, correspondant aux trajets domicile-travail.\n"
    "- Le jour le plus chargé est le mardi, mais l'activité reste élevée aussi le mercredi et le jeudi.\n"
    "- Les mois de mai à septembre affichent les volumes les plus élevés, tandis que l'hiver (décembre, janvier, février) est plus calme.\n\n"
    "Ces résultats mettent en évidence les grandes tendances de l'usage du vélo à Paris et servent de base pour les analyses détaillées dans les pages suivantes."
)
pdf.ln(5)
pdf.cell(0,10,"Les graphiques interactifs sont disponibles en cliquant sur le lien ci-dessous :")
pdf.ln(10)
# Lien cliquable
url_app = "https://testappvelo-3tskhjcmgulxgj47aa4rnp.streamlit.app/"
pdf.set_text_color(0, 0, 255)
pdf.set_font("Helvetica", 'U', 12)
pdf.cell(0, 10, "Graphiques interactifs", ln=1, link=url_app,align="C")

pdf.set_text_color(0, 0, 0)
# --- PARTIE 1 : Analyse par heure ---
pdf.add_page()
# Titre
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "1 . Analyse par heure", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.image("moyenne_heure.png", x=30, w=150)
pdf.ln(15)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "Le trafic cycliste est très faible pendant la nuit, quasi inexistant.\n\n"
    "Dès 7h, l'activité commence à augmenter rapidement, culminant entre 8h et 9h avec un premier pic marqué."
    "Au cours de la journée, le trafic reste relativement stable et modéré, avant de connaître un second pic important en fin d'après-midi, entre 17h et 19h.\n\n "
    "Ces pics correspondent principalement aux déplacements quotidiens des usagers entre leur domicile et leur lieu de travail, illustrant le rôle du vélo dans la mobilité urbaine quotidienne."
)

# --- PARTIE 2 : Analyse par jour ---
pdf.add_page()
# Titre
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "2 . Analyse par jour", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.image("moyenne_jour.png", x=30, w=150)
pdf.ln(15)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "L'activité cycliste varie au cours de la semaine, avec un trafic globalement plus important en semaine. "
    "Le mardi enregistre le volume le plus élevé, mais le mercredi et le jeudi restent très proches, avec seulement de faibles écarts. "
    "Cette répartition reflète clairement l'usage du vélo pour les déplacements domicile-travail.\n\n"
    
    "Le lundi et le vendredi présentent des volumes légèrement inférieurs, ce qui pourrait s'expliquer par une plus forte pratique du télétravail ces jours-là.\n\n"
    
    "Le samedi présente une activité intéressante, probablement liée aux sorties de loisirs et aux courses personnelles, tandis que le dimanche reste le jour le plus calme, indiquant un usage plus récréatif que professionnel."
)

# --- PARTIE 3 : Analyse par mois ---
pdf.add_page()
# Titre
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "3 . Analyse par mois", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.image("moyenne_mois.png", x=30, w=150)
pdf.ln(15)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "Les mois de mai à septembre affichent les niveaux de trafic les plus élevés, en lien avec des conditions météorologiques favorables et une luminosité accrue. "
    "À l'inverse, l'hiver (décembre, janvier, février) présente une baisse significative de l'activité cycliste.\n\n"
    "Il convient de rappeler que ces observations portent sur une seule année de données. "
    "Ainsi, un mois exceptionnellement ensoleillé, comme un juin particulièrement beau, pourrait accentuer le trafic observé, tandis qu'un mois pluvieux pourrait le réduire. "
    "Ces variations doivent être prises en compte pour interpréter correctement les tendances saisonnières."
)


# --- PARTIE 3 : Analyse par compteur ---
pdf.add_page()
# Titre
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "4 . Analyse géographique", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.image("carte_17h.png", x=30, w=150)
pdf.ln(15)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "L'analyse de la carte à 17h met en évidence une forte concentration du trafic cycliste "
    "dans l'hypercentre parisien, avec des volumes particulièrement élevés sur les axes centraux "
    "et le long de la Seine. On observe également des flux notables vers l'ouest (Boulogne, Neuilly, "
    "Issy-les-Moulineaux) et vers le sud (Montrouge, Ivry), traduisant l'importance des corridors "
    "d'entrée et de sortie de la capitale.\n\n"
    "A mesure que l'on s'éloigne du centre, l'intensité décroît, mais les points de comptage "
    "témoignent d'une pratique cycliste significative en périphérie immédiate. "
    "Le choix de 17h illustre clairement l'heure de pointe du soir, marquée par des flux soutenus "
    "liés aux déplacements domicile-travail. Cette répartition souligne le rôle central des "
    "infrastructures cyclables parisiennes et l'importance des connexions avec la petite couronne "
    "pour accompagner la croissance de l'usage du vélo."
)

# --- CONCLUSION ---
pdf.add_page()
# Titre
pdf.set_font("Helvetica", 'B', 20)
pdf.cell(0, 10, "Conclusion", ln=1, align="C")
pdf.ln(5)

pdf.set_draw_color(200, 200, 200)
pdf.set_line_width(0.2)
pdf.line(50, pdf.get_y(), 160, pdf.get_y())
pdf.ln(5)

pdf.set_font("Helvetica", '', 12)
pdf.multi_cell(0, 10,
    "L'analyse des données met en avant le rôle central du vélo dans la mobilité urbaine parisienne. "
    "Les flux horaires montrent des pics clairs le matin (8h-9h) et en fin d'après-midi (17h-19h), "
    "correspondant aux déplacements domicile-travail. Ces tendances sont complétées par une répartition hebdomadaire "
    "indiquant un trafic plus soutenu du mardi au jeudi, tandis que le lundi et le vendredi affichent des volumes un peu plus faibles, "
    "probablement liés à une plus forte pratique du télétravail. Les usages deviennent plus récréatifs le samedi et le dimanche.\n\n"
    
    "Les variations mensuelles soulignent une forte saisonnalité, avec un trafic plus élevé de mai à septembre et "
    "une baisse sensible en hiver. Il est important de rappeler que ces observations portent sur une seule année de données, "
    "ce qui implique que des conditions météorologiques particulières peuvent accentuer ou réduire ces tendances.\n\n"
    
    "L'analyse géographique des capteurs - à interpréter avec prudence car leur implantation reste inégale et "
    "certains arrondissements en sont dépourvus - met en évidence une concentration particulièrement"
    "forte des flux cyclistes dans le centre de Paris, notamment autour de Châtelet et des grands axes convergents. "
    "Les volumes plus modérés observés en périphérie traduisent une densité moindre, " 
    "ce qui souligne à la fois le rôle structurant du coeur de la capitale et la nécessité de développer "
    "des infrastructures adaptées pour accompagner l'usage du vélo dans les zones moins centrales. \n\n"
   
    
    "Ces éléments permettent non seulement de mieux comprendre les comportements des cyclistes parisiens, "
    "mais aussi de guider l'amélioration des infrastructures et des politiques de mobilité durable. "
    "Une des améliorations possibles serait peut-être l'ajout de nouveaux axes cyclables, "
    "à considérer en fonction des heures, des saisons et des jours.\n\n"
)

# Sauvegarde
pdf.output("rapport_velo.pdf")
print(" Rapport PDF généré avec succès : rapport_velo.pdf")





 Vérification doublons et valeurs manquantes
Nombre de doublons : 0
Valeurs manquantes restantes :
Date et heure de comptage          0
Identifiant du site de comptage    0
Identifiant du compteur            0
Nom du site de comptage            0
Comptage horaire                   0
Coordonnées géographiques          0
Heure                              0
Jour                               0
lat                                0
lon                                0
Mois                               0
Mois_num                           0
dtype: int64
 Rapport PDF généré avec succès : rapport_velo.pdf
